<a href="https://colab.research.google.com/github/shashank-r97/Predict-air-pollution-index/blob/master/Air_Polution_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!unzip "/content/drive/My Drive/Data/fc33077e-6-dataset.zip"

Archive:  /content/drive/My Drive/Data/fc33077e-6-dataset.zip
   creating: dataset/
  inflating: dataset/test.csv        
  inflating: dataset/train.csv       


In [2]:
!pip install catboost

     |████████████████████████████████| 64.4MB 45kB/s 


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
%%time
train = pd.read_csv('/content/dataset/train.csv')

CPU times: user 39.4 ms, sys: 14 ms, total: 53.4 ms
Wall time: 60.8 ms


In [0]:
X = train.drop(columns = ['air_pollution_index'])
y = train['air_pollution_index']

In [6]:
X.shape

(33750, 13)

In [0]:
def prepare_df(train):
    org_keys = train.columns.tolist()
    
    train['is_holiday'] = np.where(train['is_holiday'] == 'None', False, True)
    train['date_time'] = pd.to_datetime(train['date_time'])
    
    train.set_index('date_time', inplace = True)
    
    days = ['01d', '03d', '07d', '14d', '30d']
    
    sum_attributes = ['is_holiday', 'rain_p_h', 'snow_p_h']

    for val in sum_attributes:
        for day in days:
            train['num_'+val+'_'+day] = train.rolling(day)[val].sum()
            
    weather_types = train['weather_type'].unique().tolist()
    for weather in weather_types:
        train[weather] = np.where(train['weather_type'] == weather, 1, 0)
    
    train.drop(columns = 'weather_type', inplace = True)
    
    avg_attributes = ['humidity', 'wind_speed', 'wind_direction', 'visibility_in_miles', 'dew_point', 'temperature', 
                     'rain_p_h', 'snow_p_h', 'clouds_all', 'traffic_volume'] + weather_types

    for val in avg_attributes:
        for day in days:
            train['avg_'+val+'_'+day] = train.rolling(day)[val].mean()
            
    for column in train.columns:
        if 'avg' in column:
            train[column+'_diff'] = train[column[4:-4]] - train[column]
            
    return train.reset_index()

In [8]:
%%time
X = prepare_df(X)

CPU times: user 601 ms, sys: 80.7 ms, total: 682 ms
Wall time: 699 ms


In [9]:
X.shape

(33750, 248)

In [0]:
#X.drop(columns = ['date_time'], inplace = True)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [0]:
estimator = CatBoostRegressor(task_type="GPU",
                              devices='0:1')

In [14]:
estimator.fit(X_train, y_train)

Learning rate set to 0.105541
0:	learn: 83.6031259	total: 65.8ms	remaining: 1m 5s
1:	learn: 83.5967842	total: 121ms	remaining: 1m
2:	learn: 83.5881828	total: 168ms	remaining: 55.8s
3:	learn: 83.5811466	total: 214ms	remaining: 53.4s
4:	learn: 83.5623449	total: 272ms	remaining: 54.1s
5:	learn: 83.5549535	total: 321ms	remaining: 53.1s
6:	learn: 83.5377948	total: 355ms	remaining: 50.3s
7:	learn: 83.5187437	total: 387ms	remaining: 48s
8:	learn: 83.5070277	total: 418ms	remaining: 46.1s
9:	learn: 83.5010792	total: 450ms	remaining: 44.6s
10:	learn: 83.4910011	total: 481ms	remaining: 43.3s
11:	learn: 83.4789499	total: 515ms	remaining: 42.4s
12:	learn: 83.4639119	total: 546ms	remaining: 41.5s
13:	learn: 83.4514497	total: 579ms	remaining: 40.8s
14:	learn: 83.4466840	total: 608ms	remaining: 39.9s
15:	learn: 83.4343266	total: 638ms	remaining: 39.3s
16:	learn: 83.4267818	total: 669ms	remaining: 38.7s
17:	learn: 83.4172662	total: 701ms	remaining: 38.2s
18:	learn: 83.4066790	total: 729ms	remaining: 37

In [15]:
estimator.score(X_test, y_test)

-0.033574050679130885